In [3]:
import yfinance as yf
import pandas as pd
import warnings
import datetime
from tqdm import tqdm

# Ignore warnings
warnings.filterwarnings('ignore')

# Load stock list from Excel file
excel_file_path = 'Short_List_Twenty.xlsx'
sheet_name = 'Short_List_Twenty'

# Read the Excel file into a DataFrame
stock_list_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Extract the 'Ticker' column and convert it to a list
indian_stock_list = stock_list_df['Ticker'].tolist()

# Function to check stock details and identify profitable stocks
def check_stock(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="max", interval="1d")
    keys_to_extract = [
        'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk', 'maxAge', 'priceHint',
        'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 
        'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'payoutRatio', 'beta', 'trailingPE', 
        'forwardPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 
        'bid', 'ask', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 
        'fiftyDayAverage', 'twoHundredDayAverage', 'enterpriseValue', 'profitMargins', 'floatShares', 
        'sharesOutstanding', 'heldPercentInsiders', 'heldPercentInstitutions', 'impliedSharesOutstanding', 
        'bookValue', 'priceToBook', 'lastFiscalYearEnd', 'nextFiscalYearEnd', 'mostRecentQuarter', 
        'earningsQuarterlyGrowth', 'netIncomeToCommon', 'trailingEps', 'forwardEps', 'lastSplitFactor', 
        'lastSplitDate', 'enterpriseToRevenue', 'enterpriseToEbitda', '52WeekChange', 'SandP52WeekChange', 
        'lastDividendValue', 'lastDividendDate', 'shortName', 'longName', 'firstTradeDateEpochUtc', 
        'gmtOffSetMilliseconds', 'currentPrice', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice', 
        'targetMedianPrice', 'recommendationMean', 'recommendationKey', 'numberOfAnalystOpinions', 'totalCash', 
        'totalCashPerShare', 'ebitda', 'totalDebt', 'totalRevenue', 'debtToEquity', 'revenuePerShare', 
        'earningsGrowth', 'revenueGrowth', 'grossMargins', 'ebitdaMargins', 'operatingMargins'
    ]
    All_data = {}
    try:
        fifty_two_week_high = stock.info['fiftyTwoWeekHigh']
    except KeyError:
        fifty_two_week_high = 0

    if hist.empty or len(hist) < 2:
        return None

    current_price = hist['Close'][-1]
    all_time_high_till_yesterday = hist['High'][:-1].max()
    todays_high = hist['High'][-1]
    todays_open = hist['Open'][-1]

    breakout_level = fifty_two_week_high if all_time_high_till_yesterday > fifty_two_week_high and fifty_two_week_high != 0 else all_time_high_till_yesterday

    if 50 < current_price < 3000 and todays_high >= breakout_level + 1:
        volume = hist['Volume'][-1]

        if todays_open > breakout_level:
            entry_price = todays_open
            stop_loss = breakout_level - 1
        else:
            entry_price = breakout_level
            stop_loss = max(todays_open, breakout_level * 0.99) - 1

        if entry_price > 100:
            Trailing_Stop_Loss = todays_high - 5
        else:
            Trailing_Stop_Loss = entry_price - (todays_high * 0.05)

        exit_price = max(current_price, stop_loss)
        stop_loss_hit = "Yes" if current_price < stop_loss else "No"
        profit = exit_price - entry_price
        extracted_values = {key: stock.info.get(key) for key in keys_to_extract}
        All_data.update(extracted_values)

        data = {
            'Ticker': ticker,
            'All Time High': all_time_high_till_yesterday,
            '52 Week High': fifty_two_week_high,
            'Breakout Level': breakout_level,
            'Today\'s High': todays_high,
            'Today\'s Open': todays_open,
            'Current Market Price': current_price,
            'Stop Loss': stop_loss,
            'Stop Loss Points': entry_price - stop_loss,
            'Stop Loss Hit': stop_loss_hit,
            'Entry Price': entry_price,
            'Exit Price': exit_price,
            'Trailing Exit': Trailing_Stop_Loss,
            'Profit': profit,
            'Trailing Profit': Trailing_Stop_Loss - entry_price
        }
        All_data.update(data)

    return All_data

# List to store filtered stocks
filtered_stocks = []

# Loop through the tickers and apply the filter with a progress bar
for ticker in tqdm(indian_stock_list, desc="Processing stocks"):
    result = check_stock(ticker)
    if result:
        filtered_stocks.append(result)

# Create a DataFrame from the results
filtered_stocks_df = pd.DataFrame(filtered_stocks)

# Sort the DataFrame by Volume in descending order
filtered_stocks_df = filtered_stocks_df.sort_values(by='Profit', ascending=False)

# Calculate the total profit and other metrics
total_profit = filtered_stocks_df['Profit'].sum()
total_trailing_sl = filtered_stocks_df['Trailing Exit'].sum()
total_entry_capital = filtered_stocks_df['Entry Price'].sum()
total_return_percentage = (total_profit / total_entry_capital) * 100
leverage_capital = total_entry_capital * 0.2
total_return_with_leverage = (total_profit / leverage_capital) * 100

# Print the DataFrame and calculated metrics
if not filtered_stocks_df.empty:
    print(filtered_stocks_df)
    print(f"\nToday's Profit: {total_profit}")
    print(f"\nTrailing Stop Loss High Profit: {total_trailing_sl}")
    print(f"\nTotal Capital Used: {total_entry_capital}")
    print(f"\nTotal Return for the Day: {total_return_percentage:.2f}%")
    print(f"\nWith 80% Leverage, Capital Used: {leverage_capital}")
    print(f"\nTotal Return for the Day with Leverage: {total_return_with_leverage:.2f}%")

    timestamp = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M")
    filtered_stocks_df.to_csv(f'strategy_results_{timestamp}.csv', index=False)
else:
    print("No stocks meet the criteria.")

Processing stocks: 100%|██████████| 836/836 [06:20<00:00,  2.20it/s]

    auditRisk  boardRisk  compensationRisk  shareHolderRightsRisk  \
32        NaN        NaN               NaN                    NaN   
64        NaN        NaN               NaN                    NaN   
48        9.0        2.0               1.0                    5.0   
61        NaN        NaN               NaN                    NaN   
55        NaN        NaN               NaN                    NaN   
..        ...        ...               ...                    ...   
71        NaN        NaN               NaN                    NaN   
52        NaN        NaN               NaN                    NaN   
17        NaN        NaN               NaN                    NaN   
38        NaN        NaN               NaN                    NaN   
72        NaN        NaN               NaN                    NaN   

    overallRisk  maxAge  priceHint  previousClose     open   dayLow  ...  \
32          NaN   86400        2.0        1741.45  1732.85  1716.80  ...   
64          NaN   8